In [8]:
import re
from collections import Counter
import pandas as pd
import nltk
from nltk.corpus import stopwords
from IPython.display import display

# Download stopwords
nltk.download('stopwords')

# Mengambil stopwords dalam bahasa Indonesia
stopwords_indonesia = stopwords.words('indonesian')

# Data Emosi (dataset yang sudah ada)
data_emosi = {
    "level_emosi": "Sangat Buruk",
    "tipe_emosi": "Kecewa",
    "sumber_emosi": "Sekolah",
    "predicted_questions": [
        "Apa yang membuat kamu merasa kecewa di Sekolah?",
        "Apa yang bisa menyebabkan perasaan kecewa kamu di Sekolah?",
        "Bagaimana situasi di Sekolah mempengaruhi perasaan kamu yang kecewa?",
        "Bagaimana perasaan sangat buruk ini mempengaruhi keseharianmu di Sekolah?",
        "Bagaimana kamu bisa mengatasi rasa kecewa setelah situasi yang tidak sesuai harapan di Sekolah?",
        # Tambahkan pertanyaan tambahan jika diperlukan
    ],
    "user_provided_questions": [
        "Tidak mencapai nilai yang diharapkan, kesulitan memahami materi pelajaran, atau merasa tertinggal dari teman-teman",
        "Baik dari orang tua, guru, atau diri sendiri untuk berprestasi",
        "Situasi di sekolah yang negatif, seperti perundungan, diskriminasi, atau lingkungan belajar yang tidak kondusif, dapat memperparah perasaan kecewa. Sebaliknya, lingkungan sekolah yang positif dan suportif dapat membantu seseorang mengatasi perasaan kecewa.",
        "Kehilangan minat untuk belajar dan mengerjakan tugas.",
        "Berbagi perasaan dengan orang tua, guru, teman, atau konselor.",
        # Tambahkan pertanyaan tambahan yang sesuai
    ]
}

# Daftar kata kunci untuk masing-masing emosi
emosi_keywords = {
    "Kecewa": ["kecewa", "tidak dihargai", "tidak diakui", "terabaikan", "tidak puas", "tidak mencapai"],
    "Frustrasi": ["frustrasi", "kesal", "tertekan", "jengkel", "geram"],
    "Bingung": ["bingung", "kebingungan", "confused", "terperanjat", "tersesat"],
    "Bahagia": ["bahagia", "senang", "gembira", "senyum", "positif", "tersenyum"],
    "Cemas": ["cemas", "khawatir", "panik", "tertekan", "gelisah"],
    "Marah": ["marah", "kesal", "jengkel", "frustrasi", "geram"],
    "Kesal": ["kesal", "frustrasi", "marah", "geram", "iri"],
    "Sedih": ["sedih", "murung", "terpuruk", "kecewa", "hati hancur", "tidak mencapai"],
    "Tertarik": ["tertarik", "minat", "penasaran", "ingin tahu", "berminat"],
    "Optimis": ["optimis", "penuh harapan", "positif", "percaya diri", "bersemangat"],
    "Tenang": ["tenang", "damai", "relaks", "sejuk", "santai"],
    "Gembira": ["gembira", "senang", "bahagia", "bersemangat", "ceria"],
    "Puas": ["puas", "terpenuhi", "legawa", "bahagia", "senang"],
    "Nyaman": ["nyaman", "betah", "tentram", "senang", "aman"],
    "Bergairah": ["bersemangat", "bergairah", "terinspirasi", "bertekad", "penuh gairah"],
    "Aneh": ["aneh", "ganjil", "luar biasa", "tidak biasa", "unik"],
    "Euforis": ["euforis", "bersemangat", "gembira", "penuh energi", "jubile"],
    "Tertantang": ["tertantang", "bersemangat", "terdorong", "terinspirasi", "tantangan"],
    "Penuh Harapan": ["penuh harapan", "optimis", "percaya diri", "berharap", "terinspirasi"],
    "Terinspirasi": ["terinspirasi", "tergerak", "terdorong", "terpacu", "motivasi"],
    "Panik": ["panik", "cemas", "khawatir", "takut", "bingung"],
    "Rindu": ["rindu", "merindukan", "terasa kosong", "kangen", "kerinduan"],
    "Takut": ["takut", "cemas", "khawatir", "bingung", "tertekan"],
    "Senyum": ["senyum", "tersenyum", "bahagia", "gembira", "positif"],
    "Terharu": ["terharu", "tersentuh", "terpukau", "terimpressed", "terpesona"],
    "Bersyukur": ["bersyukur", "terima kasih", "berterima kasih", "berkat", "syukur"],
    "Malu": ["malu", "canggung", "memalukan", "terhina", "dipermalukan"],
    "Tersenyum": ["tersenyum", "senyum", "bahagia", "gembira", "positif"],
    "Terkejut": ["terkejut", "kaget", "terheran", "terperanjat", "bingung"],
    "Gugup": ["gugup", "cemas", "tertekan", "nervous", "khawatir"],
    "Gemas": ["gemas", "tergesa-gesa", "tertekan", "terpuruk", "cemas"],
    "Terluka": ["terluka", "terluka", "terpuruk", "tergesa-gesa", "tertekan"],
}

# Fungsi untuk membersihkan teks (menghapus stopwords dan tanda baca)
def preprocess_text(text):
    # Mengubah kalimat menjadi huruf kecil
    text = text.lower()
    
    # Menghapus tanda baca dan angka
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenisasi dan menghapus stopwords
    words = text.split()
    processed_words = [word for word in words if word not in stopwords_indonesia]
    
    return ' '.join(processed_words)

# Fungsi untuk menganalisis emosi dan mengembalikan beberapa emosi dominan
def analisis_emosi(data, top_n=3):
    user_questions = data['user_provided_questions']
    emosi_counter = Counter()

    for idx, question in enumerate(user_questions, 1):
        question_processed = preprocess_text(question)
        
        for emosi, keywords in emosi_keywords.items():
            for keyword in keywords:
                keyword_processed = preprocess_text(keyword)
                keyword_words = keyword_processed.split()
                
                # Debugging: Periksa apakah keyword_words kosong
                if not keyword_words:
                    print(f"Debug: Kata kunci '{keyword}' setelah preprocessing menjadi kosong.")
                    continue  # Skip kata kunci kosong
                
                # Jika kata kunci multi-kata, cek apakah frasa tersebut ada dalam pertanyaan
                if len(keyword_words) > 1:
                    keyword_phrase = ' '.join(keyword_words)
                    if keyword_phrase in question_processed:
                        emosi_counter[emosi] += 1
                else:
                    if keyword_words[0] in question_processed.split():
                        emosi_counter[emosi] += 1

    # Menentukan beberapa emosi dominan berdasarkan yang paling sering muncul
    if emosi_counter:
        dominant_emotions = [emosi for emosi, count in emosi_counter.most_common(top_n)]
    else:
        dominant_emotions = [data['tipe_emosi']]  # Jika tidak ada emosi yang terdeteksi

    return dominant_emotions

# Dictionary untuk rekomendasi berdasarkan emosi
emosi_rekomendasi = {
    "Kecewa": "Coba berbicara dengan teman atau konselor untuk mengatasi perasaan kecewa.",
    "Cemas": "Lakukan teknik relaksasi seperti pernapasan dalam atau meditasi.",
    "Marah": "Cari aktivitas fisik seperti olahraga untuk melepaskan ketegangan.",
    "Bahagia": "Teruskan kegiatan yang membuat Anda bahagia dan berbagi kebahagiaan dengan orang lain.",
    "Frustrasi": "Coba teknik manajemen stres atau berbicara dengan seseorang yang dipercaya.",
    "Bingung": "Cari informasi lebih lanjut atau konsultasikan dengan ahli untuk mengatasi kebingungan.",
    "Sedih": "Luangkan waktu untuk diri sendiri dan berbicara dengan orang terdekat tentang perasaan Anda.",
    "Optimis": "Tetap fokus pada tujuan dan percayalah bahwa Anda dapat mengatasi tantangan.",
    "Tenang": "Praktikkan meditasi atau aktivitas yang menenangkan untuk menjaga ketenangan.",
    "Gembira": "Nikmati momen bahagia dan bagikan kegembiraan Anda dengan orang lain.",
    "Puas": "Rayakan pencapaian Anda dan terus pertahankan kepuasan tersebut.",
    "Nyaman": "Teruskan aktivitas yang membuat Anda merasa nyaman dan aman.",
    "Bergairah": "Manfaatkan semangat Anda untuk mencapai tujuan yang diinginkan.",
    "Aneh": "Coba eksplorasi hal-hal baru yang membuat Anda merasa unik.",
    "Euforis": "Gunakan energi positif Anda untuk berkontribusi pada proyek yang bermakna.",
    "Tertantang": "Terlibat dalam tantangan baru yang dapat mengembangkan keterampilan Anda.",
    "Penuh Harapan": "Tetaplah berpegang pada harapan Anda dan terus berusaha mencapai impian.",
    "Terinspirasi": "Gunakan inspirasi Anda untuk memotivasi diri dan orang di sekitar Anda.",
    "Panik": "Segera cari bantuan profesional untuk mengatasi perasaan panik.",
    "Rindu": "Cari cara untuk terhubung kembali dengan orang yang Anda rindukan.",
    "Takut": "Identifikasi sumber ketakutan Anda dan cari strategi untuk mengatasinya.",
    "Senyum": "Teruskan kebiasaan positif ini dan bagikan senyum Anda kepada orang lain.",
    "Terharu": "Hargai momen emosional ini dan bagikan perasaan Anda dengan orang terdekat.",
    "Bersyukur": "Teruskan praktik rasa syukur dan tunjukkan apresiasi kepada orang di sekitar Anda.",
    "Malu": "Bangun kepercayaan diri dengan berlatih berbicara di depan umum atau aktivitas sosial lainnya.",
    "Tersenyum": "Teruskan kebiasaan positif ini dan bagikan senyum Anda kepada orang lain.",
    "Terkejut": "Refleksikan pengalaman tersebut dan gunakan untuk pembelajaran pribadi.",
    "Gugup": "Coba teknik relaksasi atau persiapan lebih untuk mengurangi gugup.",
    "Gemas": "Terlibat dalam aktivitas yang menyenangkan dan membangkitkan rasa gemas.",
    "Terluka": "Bicarakan perasaan Anda dengan orang yang dipercaya atau profesional kesehatan mental.",
}

# Fungsi untuk mendapatkan rekomendasi berdasarkan emosi
def get_rekomendasi(dominant_emotions):
    rekomendasi = []
    for emosi in dominant_emotions:
        rekom = emosi_rekomendasi.get(emosi, "Tidak ada rekomendasi yang tersedia untuk emosi ini.")
        rekomendasi.append(f"- {emosi}: {rekom}")
    return "\n".join(rekomendasi)

# Menjalankan analisis untuk 3 emosi dominan
hasil_analisis = analisis_emosi(data_emosi, top_n=3)

# Membuat tabel untuk Predicted Questions dan User-Provided Questions
# Pastikan jumlah elemen sama untuk menghindari ValueError
if len(data_emosi["predicted_questions"]) == len(data_emosi["user_provided_questions"]):
    questions_df = pd.DataFrame({
        "Predicted Questions": data_emosi["predicted_questions"],
        "User Responses": data_emosi["user_provided_questions"]
    })

    # Membuat tabel untuk Ringkasan Emosi
    summary_df = pd.DataFrame({
        "Level Emosi": [data_emosi["level_emosi"]],
        "Tipe Emosi": [data_emosi["tipe_emosi"]],
        "Sumber Emosi": [data_emosi["sumber_emosi"]],
        "Dominant Emotions": [", ".join(hasil_analisis)]  # Gabungkan hasil analisis menjadi string
    })

    # Menampilkan Tabel
    print("Tabel Predicted Questions dan User Responses:")
    display(questions_df)

    print("\nRingkasan Emosi Dominan:")
    display(summary_df)

    # Mendapatkan rekomendasi
    recommendations = get_rekomendasi(hasil_analisis)

    # Menampilkan rekomendasi
    print("\nRekomendasi Berdasarkan Emosi Dominan:")
    print(recommendations)
else:
    print(f"Error: Jumlah 'predicted_questions' ({len(data_emosi['predicted_questions'])}) dan 'user_provided_questions' ({len(data_emosi['user_provided_questions'])}) tidak sama. Pastikan kedua daftar memiliki jumlah elemen yang sama.")
p

Debug: Kata kunci 'ingin tahu' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'luar biasa' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'tidak biasa' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'ingin tahu' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'luar biasa' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'tidak biasa' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'ingin tahu' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'luar biasa' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'tidak biasa' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'ingin tahu' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'luar biasa' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'tidak biasa' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'ingin tahu' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'luar biasa' setelah preprocessing menjadi kosong.
Debug: Kata kunci 'tidak biasa

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucifrr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Predicted Questions,User Responses
0,Apa yang membuat kamu merasa kecewa di Sekolah?,"Tidak mencapai nilai yang diharapkan, kesulita..."
1,Apa yang bisa menyebabkan perasaan kecewa kamu...,"Baik dari orang tua, guru, atau diri sendiri u..."
2,Bagaimana situasi di Sekolah mempengaruhi pera...,"Situasi di sekolah yang negatif, seperti perun..."
3,Bagaimana perasaan sangat buruk ini mempengaru...,Kehilangan minat untuk belajar dan mengerjakan...
4,Bagaimana kamu bisa mengatasi rasa kecewa sete...,"Berbagi perasaan dengan orang tua, guru, teman..."



Ringkasan Emosi Dominan:


,Level Emosi,Tipe Emosi,Sumber Emosi,Dominant Emotions
0,Sangat Buruk,Kecewa,Sekolah,"Kecewa, Sedih, Bahagia"



Rekomendasi Berdasarkan Emosi Dominan:
- Kecewa: Coba berbicara dengan teman atau konselor untuk mengatasi perasaan kecewa.
- Sedih: Luangkan waktu untuk diri sendiri dan berbicara dengan orang terdekat tentang perasaan Anda.
- Bahagia: Teruskan kegiatan yang membuat Anda bahagia dan berbagi kebahagiaan dengan orang lain.
